In [1]:
""" 
dependencies:
  - python=3.8.17
  - numpy=1.24.0
  - matplotlib=3.7.1
  - pandas=2.0.2 
"""
import os
import random
import datetime
from itertools import product 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# import random
# random.seed(42)
# np.random.seed(42)
# np.random.RandomState(42)
# os.environ['TF_DETERMINISTIC_OPS'] = '1' 

finish_sound = "afplay /Users/mehmet/Documents/vs-code/winsquare.mp3"
# play sound when finished
# os.system(finish_sound)

In [2]:
# Read data from npy file ( already preprocessed )
filename = 'original-numpy'
# filename = 'pca-numpy'
X_train = np.load(f'dataset/{filename}/X_train.npy')
X_val = np.load(f'dataset/{filename}/X_val.npy')
X_test = np.load(f'dataset/{filename}/X_test.npy')
y_train = np.load(f'dataset/{filename}/y_train.npy')
y_val = np.load(f'dataset/{filename}/y_val.npy')
y_test = np.load(f'dataset/{filename}/y_test.npy')

# # Add Bias to X
# X_train = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
# X_val = np.concatenate((np.ones((X_val.shape[0], 1)), X_val), axis=1)
# X_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1)
# X_train.shape, X_val.shape, X_test.shape

# # Remove one hot encoding from y
# y_train = np.argmax(y_train, axis=1)
# y_val = np.argmax(y_val, axis=1)
# y_test = np.argmax(y_test, axis=1)

print(X_train.shape, y_train.shape,'\n', X_val.shape, y_val.shape,'\n', X_test.shape, y_test.shape)



(5120, 10859) (5120, 4) 
 (640, 10859) (640, 4) 
 (640, 10859) (640, 4)


In [3]:
class SVM:
    def __init__(self, seed=42):
        np.random.seed(seed)
        self.W = None
        self.now = None
        self.print_result = True
        self.history_steps1 = None
        self.history = None
        self.validation_accuracy = None
        
    def validation_accuracy(self):
        return self.validation_accuracy
    
    def history(self):
        return self.history
    
    def load_history(self):
        pd_hist = pd.read_csv(f'model-comparison/{self.now}/history.csv')
        self.history = np.array(pd_hist.iloc[:,1:])
        
    def plot(self, save = True):
        # Save history as csv file
        history_local = self.history
        if type(history_local) is not pd.DataFrame:
            history_df = pd.DataFrame(history_local)
        if save == True:
            hist_csv_file = f'model-comparison/{self.now}/history.csv'
            with open(hist_csv_file, mode='w') as f:
                history_df.to_csv(f) 
        # Plot Loss and Accuracy History as Subplots
        fig, ax = plt.subplots(1, 2)
        fig.set_size_inches(10, 2)
        index = np.arange(1,self.history.shape[1]+1)*self.history_steps1

        ax[0].plot(index, self.history[0], label='Training Loss')
        ax[0].plot(index, self.history[2], label='Validation Loss')
        ax[0].set_title('Loss History')
        ax[0].set_xlabel('Epoch')
        ax[0].set_ylabel('Loss')
        ax[0].legend()

        ax[1].plot(index, self.history[1], label='Training Accuracy')
        ax[1].plot(index, self.history[3], label='Validation Accuracy')
        ax[1].set_title('Accuracy History')
        ax[1].set_xlabel('Epoch')
        ax[1].set_ylabel('Accuracy')
        ax[1].legend()
        if save is True and self.now is not None:
            plt.savefig(f'model-comparison/{self.now}/plot.png')
        if self.print_result == True:
            plt.show()
        else:
            plt.close(fig)
        

    def loss(self, X_nonbiased, y, W):
        # Hinge loss
        if X_nonbiased.shape[1] != W.shape[0]:
            ones=np.ones(X_nonbiased.shape[0])
            X=np.c_[ones,X_nonbiased]
        else:
            X = X_nonbiased
        scores = X.dot(W)
        num_samples = X.shape[0]
        correct_class_mask = (np.arange(num_samples), y)
        margins = np.maximum(0, scores - scores[correct_class_mask][:, np.newaxis] + 1)
        margins[correct_class_mask] = 0
        loss = np.sum(margins)
        
        return loss
    
    def gradient(self, X, y, W):
        scores = X.dot(W)
        num_samples = X.shape[0]
        correct_class_mask = (np.arange(num_samples), y)
        margins = np.maximum(0, scores - scores[correct_class_mask][:, np.newaxis] + 1)
        margins[correct_class_mask] = 0
        grad_mask = (margins > 0).astype(float)
        grad_mask[correct_class_mask] = -np.sum(grad_mask, axis=1)
        return grad_mask
        
    def fit(self, X_nonbiased, y, X_val, y_val, now=None, print_result = True,
            batch_size=5120, weight_init='zero', lr=0.01, lr_type = 'static', lmbda=0.01, max_epoch=1000,
            history_steps = 50, print_step = 100):
        start_time = datetime.datetime.now()
        # if there isn't model-comparison folder, create it
        if not os.path.exists('model-comparison'):
            os.mkdir('model-comparison')
        self.print_result = print_result
        if now is not None:
            self.now = now
        # Create folder for current model
            if not os.path.exists('model-comparison/'+now):
                os.mkdir('model-comparison/'+now)

        self.history_steps1 = history_steps
        self.history = np.zeros((4,max_epoch//history_steps))
        y_onehot = y
        lr_print = str(lr) + ' ' + lr_type
        model_specs = 'SVM | Batch Size: {} | Weight Init. {} | lr: {} | Lambda: {} | Max Epoch: {} |'.format(batch_size, weight_init, lr_print, lmbda, max_epoch)

        # add bias
        ones=np.ones(X_nonbiased.shape[0])
        X=np.c_[ones,X_nonbiased]
        
        num_classes = y.shape[1]
        num_samples, num_features = X.shape

        #self.W = np.random.rand(num_features, num_classes)
        
        # zero initialization
        # bias included in W
        self.W = np.zeros((num_features, num_classes))

        # One hot encoded to not one hot encoded
        y = np.argmax(y, axis=1)
        y_val = np.argmax(y_val, axis=1)
        
        # Print loss and accuracy every 100 iterations or every max_iter//10 iterations if max_iter >= 1000
        if max_epoch >= 1000:
            print_step = max_epoch // 10

        # Gradient Descent
        for epoch in range(1,max_epoch+1):
            
            # Shuffle all data X and y in the same order every epoch
            shuffle_index = np.arange(X.shape[0])
            np.random.shuffle(shuffle_index)
            X = X[shuffle_index]
            y = y[shuffle_index]
            
            for iteration in range(X.shape[0]//batch_size):          
                                              
                X_batch = X[batch_size*iteration:batch_size*(iteration+1)]
                y_batch = y[batch_size*iteration:batch_size*(iteration+1)]
            
                reg_term = lmbda * self.W
                reg_term[0] = 0
                
            
                gradient = self.gradient(X_batch, y_batch, self.W)
                
                if lr_type[0:8] == 'momentum':
                    if epoch == 1:
                        last_gradient = gradient
                    else:
                        momentum = float(lr_type[10:])
                        gradient = gradient + momentum * last_gradient
                        last_gradient = gradient
                
                
                self.W -= lr * (X_batch.T.dot(gradient) / batch_size + reg_term)
                
            if lr_type[0:8] == 'adaptive' and epoch % 500 == 0:
                    k = float(lr_type[9:])
                    lr *= k  

            # For each 100 epochs print losses and accuracy
            if epoch % history_steps == 0:
                # how to calculate accuracy
                loss = self.loss(X, y, self.W)
                val_loss = self.loss(X_val, y_val, self.W)
                accuracy = np.mean(self.predict(X) == y)  
                val_acc = np.mean(self.predict(X_val) == y_val)   
                self.validation_accuracy = val_acc
                self.history[:,(epoch//history_steps)-1] = np.array([loss, accuracy, val_loss, val_acc])
                
                if epoch % print_step == 0:
                    line1 = 'Epoch: ' + str(epoch)
                    line2 = ' | Loss: ' + str(loss)[:5] + ' | Accuracy: ' + str(accuracy)[0:5]
                    line3 = ' | Val. Loss: ' + str(val_loss)[:5] + ' | Val. Acc: ' + str(val_acc)[0:5]
                    # line2 = ' | Loss: ' + str(round(loss)) + ' | Accuracy: ' + str(accuracy)[0:5]
                    # line3 = ' | Val. Loss: ' + str(round(val_loss)) + ' | Val. Acc: ' + str(val_acc)[0:5]
                    if print_result == True:
                        print(line1 + line2 + line3)
                    if now is not None:
                        with open('model-comparison/{}/log.txt'.format(now), 'a') as f:
                            f.write(line1 + line2 + line3 + '\n')
                            
            if epoch == max_epoch:
                end_time = datetime.datetime.now()
                if print_result == True:
                    print('Training finished. Time elapsed:', end_time - start_time, '\n')
                    print('Accuracy: ', str(accuracy)[0:5], 'Val. Accuracy: ', str(val_acc)[0:5])
                val_acc_print = str(val_acc*100)+ '00'
                if now is not None:
                    with open('model-comparison/{}/log.txt'.format(now), 'a') as f:
                        write_line = 'Training finished. Time elapsed: ' + str(end_time - start_time) + '\n'
                        f.write(write_line)
                    with open('model-comparison/{}/{}-val-acc.txt'.format(now,val_acc_print[0:5]), 'w') as f:
                        f.write(model_specs)
                    with open('model-comparison/last.txt', 'w') as f:
                        f.write(str(now))
                            
    def predict(self, X_nonbiased):
        if X_nonbiased.shape[1] != self.W.shape[0]: 
            # add bias
            ones=np.ones(X_nonbiased.shape[0])
            X=np.c_[ones,X_nonbiased]
        else:
            X = X_nonbiased
        
        scores = X.dot(self.W)
        predictions = np.argmax(scores, axis=1)
        return predictions
    
    def save_weights(self):
        filename = 'model-comparison/{}/weights.npy'.format(self.now)
        np.save(filename, self.W)
    def load_weights(self, now):
        filename = 'model-comparison/{}/weights.npy'.format(now)
        self.W = np.load(filename)
        self.now = now

In [4]:
class EvaluateModel():
    # Class to evaluate model performance, similar to sklearn.metrics ClassificationReport and ConfusionMatrix
    def __init__(self, y_true, y_pred, str1, now, save=True, print_result=True):
        self.y_true = np.argmax(y_true, axis=1)
        self.y_pred = y_pred
        if save == True:
            os.mkdir('model-comparison/'+now+'/'+str1)
            np.savetxt('model-comparison/{}/{}/pred.csv'.format(now,str1), y_pred, delimiter=',', fmt='%d')
        
        result = self.classification_report()
        fpr0 = 100 - float(result['precision'][0][0:4])
        line1 = 'Accuracy is: ' + str(result['f1-score']['accuracy'])
        line2 = 'F1 Score is: ' + str(result['f1-score']['weighted avg'])
        line3 = 'Precision of Class 0 is: ' + '{0:.2f}'.format(100-fpr0)+ ' %'
        line4 = '\nClassification Report:'
        line5 = '\nConfusion Matrix:'
        cm = self.confusion_matrix()
        line6 = '\n'
        res_total = line1 + '\n' + line2 + '\n' + line3 + '\n' + line4 + '\n' + str(result) + '\n' + line5 + '\n' + str(cm) + '\n' + line6
        # write to file
        if save == True:
            with open('model-comparison/{}/{}/report.txt'.format(now,str1), 'w') as f:
                f.write(res_total)
        if print_result == True:
            print(res_total)

    def accuracy_score(self, y_t, y_p):
        correct = sum(y_t == y_p)
        return correct / len(y_t)

    def scores(self, y_t, y_p, class_label= 1):
        true = y_t == class_label
        pred = y_p == class_label
        tp = sum(true & pred)
        fp = sum(~true & pred) 
        fn = sum(true & ~pred)
        tn = sum(~true & ~pred) 
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * (precision * recall) / (precision + recall)
        return precision, recall, f1
    
    def confusion_matrix(self,labels=None):
        labels = labels if labels else sorted(set(self.y_true) | set(self.y_pred))        
        indexes = {v:i for i, v in enumerate(labels)}
        matrix = np.zeros((len(indexes),len(indexes))).astype(int)
        for t, p in zip(self.y_true, self.y_pred):
            matrix[indexes[t], indexes[p]] += 1
        # print('Confusion Matrix: ')
        # print(pd.DataFrame(matrix, index=labels, columns=labels))
        return pd.DataFrame(matrix, index=labels, columns=labels)

    def classification_report(self):
        output_dict = {}
        support_list = []
        precision_list = []
        recall_list = []
        f1_list = []
        for i in np.unique(self.y_true):
            support = sum(self.y_true == i)
            precision, recall, f1 = self.scores(self.y_true, self.y_pred, class_label=i)
            output_dict[i] = {'precision':precision, 'recall':recall, 'f1-score':f1, 'support':support}
            precision_list.append(precision)
            recall_list.append(recall)
            f1_list.append(f1)
            support_list.append(support)
        support = np.sum(support_list)
        output_dict['accuracy'] = {'precision':0, 'recall':0, 'f1-score':self.accuracy_score(self.y_true, self.y_pred), 'support':support}
        # macro avg
        macro_precision = np.mean(precision_list)
        macro_recall = np.mean(recall_list)
        macro_f1 = np.mean(f1_list)
        output_dict['macro avg'] = {'precision':macro_precision, 'recall':macro_recall, 'f1-score':macro_f1, 'support':support}
        # weighted avg
        weighted_precision = np.average(precision_list, weights=support_list)
        weighted_recall = np.average(recall_list, weights=support_list)
        weighted_f1 = np.average(f1_list, weights=support_list)
        output_dict['weighted avg'] = {'precision':weighted_precision, 'recall':weighted_recall, 'f1-score':weighted_f1, 'support':support}
        # convert to dataframe and format
        report_d = pd.DataFrame(output_dict).T
        annot = report_d.copy()
        annot.iloc[:, 0:3] = (annot.iloc[:, 0:3]*100).applymap('{:.2f}'.format) + ' %'
        annot['support'] = annot['support'].astype(int)
        annot.loc['accuracy','precision'] = ''
        annot.loc['accuracy','recall'] = ''
        return annot

In [5]:
def GridSearch(model_options, X_train, y_train, X_val, y_val, X_test, y_test, print_result=False, seed=42):
    # Grid Search Function
    best_metric = 0
    for i in range(len(model_options)):
        models = model_options[i]
        model_number = i + 1
        now = datetime.datetime.now().strftime("%d-%m-%H-%M")
        # Create folder for current model
        if not os.path.exists('model-comparison/'+now):
            os.mkdir('model-comparison/'+now)
        else:
            now = now + str('--1')
            os.mkdir('model-comparison/'+now)
        model = SVM(seed=seed)
        start_time = datetime.datetime.now()
        model.fit(X_train, y_train, X_val, y_val, now, print_result=print_result, max_epoch=models[0], 
                  weight_init= models[1], batch_size=models[2], lr=models[3], lr_type=models[4], lmbda=models[5])
        end_time = datetime.datetime.now()
        time_elapsed = str(end_time - start_time)[2:7]
        metric = model.validation_accuracy
        model.save_weights()
        model.plot()
        y_pred = model.predict(X_val)
        results = EvaluateModel(y_val, y_pred, 'val', now, print_result=print_result)
        y_pred = model.predict(X_test)
        results = EvaluateModel(y_test, y_pred, 'test', now, print_result=print_result)
        if metric > best_metric:
            best_metric = metric
            best_model = now
        print('Model ', str(model_number), ' saved with name: ', now)
        print(models, 'Val-Accuracy:', metric)

        # append to txt file
        lr_print = str(models[3]) + ' ' + models[4]
        model_specs = 'SVM | Batch Size: {} | Weight Init: {} | lr: {} | Lambda: {} | Max Epoch: {}'.format(models[2], models[1], lr_print, models[5], models[0])
        with open('model-comparison/best-models.txt', 'a') as f:
            f.write(now + ' | ' + model_specs + ' | ' + str(metric) + ' | Time Elapsed: '+ time_elapsed +'\n')
        print(len(model_options)-model_number, 'models left to train.')
    best_metric = str(best_metric*100)[:5]
    print('Best Model is:', best_model, 'with validation accuracy:', best_metric, '%')

In [6]:
def TrainModel(max_epoch, batch_size, weight_init, lr, lr_type, lmbda):
    model_options = [[max_epoch, weight_init, batch_size, lr, lr_type, lmbda]]
    return model_options

In [7]:
# Train New Model
model_parameters = TrainModel(
    max_epoch=2000, batch_size=512, weight_init='zero', lr=0.0001, lr_type='adaptive 0.1', lmbda=0.001)

GridSearch(model_parameters, X_train, y_train, X_val, y_val, X_test, y_test, print_result=True, seed=42)

os.system(finish_sound)

Epoch: 200 | Loss: 1623. | Accuracy: 0.941 | Val. Loss: 524.5 | Val. Acc: 0.828
Epoch: 400 | Loss: 604.1 | Accuracy: 0.977 | Val. Loss: 399.8 | Val. Acc: 0.864
Epoch: 600 | Loss: 406.5 | Accuracy: 0.984 | Val. Loss: 369.9 | Val. Acc: 0.871
Epoch: 800 | Loss: 381.4 | Accuracy: 0.985 | Val. Loss: 366.7 | Val. Acc: 0.873
Epoch: 1000 | Loss: 358.8 | Accuracy: 0.986 | Val. Loss: 363.5 | Val. Acc: 0.876


In [71]:
# # Train New Model
# now = datetime.datetime.now().strftime("%d-%m-%H-%M")

# model = SVM(seed=42)
# model.fit(X_train, y_train, X_val, y_val, now=now, print_result=True,
#         lr=0.001, lmbda=0.001, max_epoch=1000)

# model.save_weights()
# model.plot()

# # Validation Set Results
# y_pred = model.predict(X_val)
# results = EvaluateModel(y_val, y_pred, 'val', now)

# # Test Set Results
# y_pred = model.predict(X_test)
# results = EvaluateModel(y_test, y_pred, 'test', now)


In [83]:
# Grid Search Combinations

max_epoch = [1000]
weight_init = ['zero']
#batch_size = [1, 512, 5120]
batch_size = [5120]
# lr = [0.01, 0.005, 0.001, 0.0005, 0.0001]
lr = [0.01, 0.001, 0.0001]

lr_type = ['static']
regularization = [0.01, 0.001, 0.0001, 0]
#regularization = [0.01, 0.005, 0.001, 0.0005, 0.0001]
params = [max_epoch, weight_init, batch_size, lr, lr_type, regularization]
model_options = list(product(*params))
print('Number of combinations:', len(model_options))
print('Combination 1:', model_options[0])

Number of combinations: 12
Combination 1: (1000, 'zero', 5120, 0.01, 'static', 0.01)


In [84]:
# Grid Search All Combinations

GridSearch(model_options, X_train, y_train, X_val, y_val, X_test, y_test)
os.system(finish_sound)

Model  1  saved with name:  18-12-17-34
(1000, 'zero', 5120, 0.01, 'static', 0.01) Val-Accuracy: 0.8546875
11 models left to train.
Model  2  saved with name:  18-12-17-35
(1000, 'zero', 5120, 0.01, 'static', 0.001) Val-Accuracy: 0.8515625
10 models left to train.
Model  3  saved with name:  18-12-17-37
(1000, 'zero', 5120, 0.01, 'static', 0.0001) Val-Accuracy: 0.8515625
9 models left to train.
Model  4  saved with name:  18-12-17-38
(1000, 'zero', 5120, 0.01, 'static', 0) Val-Accuracy: 0.85
8 models left to train.
Model  5  saved with name:  18-12-17-40
(1000, 'zero', 5120, 0.001, 'static', 0.01) Val-Accuracy: 0.89375
7 models left to train.
Model  6  saved with name:  18-12-17-41
(1000, 'zero', 5120, 0.001, 'static', 0.001) Val-Accuracy: 0.9
6 models left to train.
Model  7  saved with name:  18-12-17-43
(1000, 'zero', 5120, 0.001, 'static', 0.0001) Val-Accuracy: 0.9
5 models left to train.
Model  8  saved with name:  18-12-17-44
(1000, 'zero', 5120, 0.001, 'static', 0) Val-Accuracy:

0